# Vis 5
##  Data Processing

In [1]:
import altair as alt
import json
from altair import datum
from vega_datasets import data
import numpy as np
import requests
import pandas as pd
import json
from geopy.geocoders import Nominatim
import pprint
import csv
from collections import Counter
pp = pprint.PrettyPrinter(indent=4)

locs = pd.read_csv('./pleiades-locations-latest.csv')
places = pd.read_csv('./pleiades-places-latest.csv')
names = pd.read_csv('./pleiades-names-latest.csv')

capitals = pd.read_json('http://techslides.com/demos/country-capitals.json')


print("locs.shape = ",locs.shape)
print("places.shape = ",places.shape)
print("names.shape = ",names.shape)
print("capitals.shape = ",capitals.shape)

locs.shape =  (40463, 25)
places.shape =  (37737, 26)
names.shape =  (34133, 26)
capitals.shape =  (245, 6)


In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Data Prep

In [3]:
def getCountry(reprLatLong):
    geolocator = Nominatim(user_agent="my-appliation")
    location = geolocator.reverse(reprLatLong)
    local = location.address
    localCSV = local.split(',')
    return localCSV[-1]
    print(localCSV[-1])

In [4]:
settlementFilter=locs.featureType==("settlement")
settlementData = locs[settlementFilter]
settlementPoints = settlementData[1:settlementData.size:10]
settlementData.shape

(1846, 25)

In [5]:
# ## sendding data to file

settlementCoords = settlementPoints.reprLatLong
settlementCoords.tolist()[:]
pp.pprint (settlementCoords)

countryCoords = []
for i in settlementCoords:
    countryCoords.append(getCountry(i))
    
pp.pprint (countryCoords)



43             37.5966266,-4.3084564
433              34.006821,-6.821498
741                36.50978,1.308828
859      36.5945343511,2.44289145652
1112     37.6665931729,32.8282455857
                    ...             
39301             49.894591,2.302064
39405              50.40064,5.376434
39714            49.437202,10.121086
39875            48.151667,11.643889
39970    31.9509005802,45.6233775029
Name: reprLatLong, Length: 185, dtype: object


AttributeError: 'NoneType' object has no attribute 'address'

In [ ]:
# len(countryCoords)

In [ ]:
with open('1301.1846', 'wt',encoding="utf-8") as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(countryCoords)

## Visulisation

In [ ]:
# data manually copied in from outputted csv files using getCountry function with the settlemnet data
apperance=[#1-400
'Spain','Morocco','Algeria','Algeria','Turkey','France','France','Algeria','Libya','Italy','Turkey','Greece','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Turkey','Kashmir','Bulgaria','Romania','Romania','Italy','Israel','Algeria','Tunisia','Tunisia','Tunisia','Tunisia','Tunisia','Syria','Lebanon','Lebanon','Israel',
#500-900
'Egypt','Greece','Kashmir','Greece','Turkey','Turkey','Turkey','Turkey','Italy','Italy','Italy','Italy','Italy','Italy','Iraq','Greece','Greece','Greece','Turkey','Turkey','Turkey','Italy','Italy','Italy','Turkey','Turkey','Turkey','Turkey','Turkey','Syria','Turkey','Italy','Italy','Iraq','United Kingdom','United Kingdom','United Kingdom','United Kingdom','United Kingdom','Syria',
#901-1300
'Turkey','Turkey','Turkey','Turkey','Algeria','Libya','Greece','Greece','Greece','Italy','Iran','Iraq','India','Spain','Spain','Italy','Turkey','Greece','Greece','Albania','Macedonia','Armenia','Iraq','France','France','Macedonia','Greece','Greece','Greece','Italy','Turkey','Tunisia','Tunisia','France','Switzerland','France','Switzerland','Hungary','Croatia','Egypt',
#1301-1846
'Turkey','Iraq','Syria','Iraq','Iran','Afghanistan','Lebanon','Turkey','Turkey','Syria','Syria','Syria','Israel','Jordan','Germany','France','France','France','France','Portugal','Spain','United Kingdom','Italy','Romania','Iran','Turkey','Israel','France','Turkey','Italy','Italy','Italy','Italy','Italy','Italy','Portugal','Spain','France','France','Spain','Syria','Iraq','Iraq','Iran','France','Germany','France','Belgium','Germany','France','France','Belgium','Germany','Germany','Iraq']

In [ ]:
#for manual hist
apperanceDist = list(dict.fromkeys(apperance))
apperanceCount = list(Counter(apperance).values())

In [ ]:
# b = [i for i in range(len(apperance))]

# print ("list.len = ", len(apperance))
# print ("b.len = ", len(b))

In [ ]:
#auto histogram (auto filters distincy countrys and counts)
# data = pd.DataFrame({'CountryApperance': apperance})

# alt.Chart(data).mark_bar().encode(
#     x='CountryApperance',
#     y='count(CountryApperance)'
# )

In [ ]:
#manual histogram
df = pd.DataFrame({'Country': apperanceDist,
                     'Settlement Rate (x 10)': apperanceCount})

histo = alt.Chart(df).mark_bar().encode(
    x='Country',
    y='Settlement Rate (x 10)',
    tooltip=['Settlement Rate (x 10):Q'],
)

In [ ]:
matrix = np.c_[apperanceDist,apperanceCount]
matrix.shape
matrix
#matrix[:,[1]]
#matrix[:,[0]]

In [ ]:
#prerequisites
centerVal = [-20,60]
scaleVal = 350

mapData = alt.topo_feature("https://vega.github.io/vega/data/world-110m.json",'countries')
mapDataWithCountryNames= alt.topo_feature("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json",'id')
source = pd.read_csv('./choro.csv')


In [ ]:
choro= alt.Chart(source).mark_geoshape().encode(
    shape='geo:G',
    color='Settlement Rate (x 10):Q',
    tooltip=['Settlement Rate (x 10):Q'],
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=mapData, key='id'),
    as_='geo'
).properties(
    width=500,
    height=300
).project(
    center = centerVal,
    scale=scaleVal,
)


histo | (choro)